In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
MONTHS = [6,7,8,9]
SPECIES = ['CULEX PIPIENS', 'CULEX PIPIENS/RESTUANS', 'CULEX RESTUANS'] 
TRAPS = []

In [3]:
import pandas as pd


train_df = pd.read_csv('../data/raw/train.csv')
weather_df = pd.read_csv('../data/raw/weather.csv')

In [4]:
from src.data import clean_weather

weather_df = clean_weather(weather_df)

In [5]:
from src.data import remove_address, split_date

train_df = remove_address(split_date(train_df))

In [6]:
train_df = train_df[train_df.Month.isin(MONTHS)]
train_df = train_df[train_df.Species.isin(SPECIES)]

In [7]:
trap_max_mos = train_df.groupby(['Date', 'Trap'])['NumMosquitos'].sum().reset_index().groupby(['Trap'])['NumMosquitos'].max()

trap_prob = train_df.groupby(['Date', 'Trap'])['WnvPresent'].max().reset_index().groupby(['Trap'])['WnvPresent'].mean()

In [8]:
import numpy as np


trap_max_mos.corr(trap_prob)

np.float64(0.4243543847666627)

In [9]:
single_row = train_df.groupby(
    ['Date', 'Trap', 'Species']
).agg(
    RowCount=('NumMosquitos', 'count'),
    NumMosquitos=('NumMosquitos', 'sum'),
    WnvPresent=('WnvPresent', 'max')
).reset_index()

single_row

,Date,Trap,Species,RowCount,NumMosquitos,WnvPresent
0,2007-06-05,T002,CULEX PIPIENS,1,1,0
1,2007-06-05,T002,CULEX PIPIENS/RESTUANS,1,3,0
2,2007-06-05,T002,CULEX RESTUANS,1,5,0
3,2007-06-05,T009,CULEX RESTUANS,1,2,0
4,2007-06-05,T011,CULEX RESTUANS,1,1,0
...,...,...,...,...,...,...
7832,2013-09-26,T232,CULEX PIPIENS/RESTUANS,1,1,0
7833,2013-09-26,T233,CULEX PIPIENS/RESTUANS,1,5,0
7834,2013-09-26,T235,CULEX PIPIENS/RESTUANS,1,1,0
7835,2013-09-26,T900,CULEX PIPIENS,4,37,0


In [10]:
single_row.WnvPresent.mean()

np.float64(0.058057930330483606)

In [11]:
single_row.RowCount.max()

np.int64(51)

In [12]:
single_row.NumMosquitos.max()

np.int64(2532)

In [13]:
single_row[['RowCount', 'NumMosquitos']].corr()

,RowCount,NumMosquitos
RowCount,1.000000,0.962729
NumMosquitos,0.962729,1.000000


In [66]:
train_df.columns

Index(['Date', 'Species', 'Trap', 'Latitude', 'Longitude', 'NumMosquitos',
       'WnvPresent', 'Month', 'Year', 'Week', 'Dayofyear', 'Dayofweek'],
      dtype='object')

In [17]:
single_row.columns

Index(['Date', 'Trap', 'Species', 'RowCount', 'NumMosquitos', 'WnvPresent'], dtype='object')

In [24]:
train_df = single_row.merge(
    train_df[
        ['Date', 'Species', 'Trap', 'Latitude', 'Longitude', 'Month', 'Year', 'Week', 'Dayofyear', 'Dayofweek']
    ].drop_duplicates(),
    on=['Date', 'Species', 'Trap'],
    how='left'
)

In [26]:
train_df

,Date,Trap,Species,RowCount,NumMosquitos,WnvPresent,Latitude,Longitude,Month,Year,Week,Dayofyear,Dayofweek
0,2007-06-05,T002,CULEX PIPIENS,1,1,0,41.954690,-87.800991,6,2007,23,156,1
1,2007-06-05,T002,CULEX PIPIENS/RESTUANS,1,3,0,41.954690,-87.800991,6,2007,23,156,1
2,2007-06-05,T002,CULEX RESTUANS,1,5,0,41.954690,-87.800991,6,2007,23,156,1
3,2007-06-05,T009,CULEX RESTUANS,1,2,0,41.981964,-87.812827,6,2007,23,156,1
4,2007-06-05,T011,CULEX RESTUANS,1,1,0,41.944869,-87.832763,6,2007,23,156,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7832,2013-09-26,T232,CULEX PIPIENS/RESTUANS,1,1,0,41.912563,-87.668055,9,2013,39,269,3
7833,2013-09-26,T233,CULEX PIPIENS/RESTUANS,1,5,0,42.009876,-87.807277,9,2013,39,269,3
7834,2013-09-26,T235,CULEX PIPIENS/RESTUANS,1,1,0,41.776428,-87.627096,9,2013,39,269,3
7835,2013-09-26,T900,CULEX PIPIENS,4,37,0,41.974689,-87.890615,9,2013,39,269,3
